# Step 1: import 必要套件

In [ ]:
from main.model.squeezeDet import SqueezeDet
from main.model.dataGenerator import generator_from_data_path, visualization_generator_from_data_path
from main.model.visualization import visualize
from main.config.create_config import load_dict
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import pickle
import os
import gc

# Step 2: 基本設定 & 載入檔案
* img_train.txt
* gt_train.txt
* squeeze.config

In [ ]:
img_file = "img_train.txt"
gt_file  = "gt_train.txt"
log_dir_name = './log'
EPOCHS = 100
STEPS = None
OPTIMIZER = "default"
REDUCELRONPLATEAU = True
VERBOSE = True
CONFIG = "squeeze.config"

tb_dir = log_dir_name +"/tensorboard"

if "log" not in os.listdir():
    os.makedirs(tb_dir)

In [ ]:
# 目前有 img_names 與 gt_names 還沒有 sorted 的情況
with open(img_file) as imgs:
    img_names = imgs.read().splitlines()
imgs.close()

In [ ]:
with open(gt_file) as gts:
    gt_names = gts.read().splitlines()
gts.close()

In [ ]:
cfg = load_dict(CONFIG)

cfg.img_file = img_file
cfg.gt_file = gt_file
cfg.images = img_names
cfg.gts = gt_names
#cfg.init_file = init_file
cfg.EPOCHS = EPOCHS
cfg.OPTIMIZER = OPTIMIZER
#cfg.CUDA_VISIBLE_DEVICES = CUDA_VISIBLE_DEVICES
#cfg.GPUS = GPUS
cfg.REDUCELRONPLATEAU = REDUCELRONPLATEAU

In [ ]:
train_generator = generator_from_data_path(img_names, gt_names, config=cfg)
vis_train_generator = visualization_generator_from_data_path(img_names, gt_names, config=cfg)

In [ ]:
# compute number of batches per epoch
nbatches_train, mod = divmod(len(img_names), cfg.BATCH_SIZE)
print(f"Number of images: {len(img_names)}")
print(f"Number of epochs: {EPOCHS}")
print(f"Number of batches: {nbatches_train}")
print(f"Batch size: {cfg.BATCH_SIZE}")

In [ ]:
# instantiate model
squeeze = SqueezeDet(cfg)

In [ ]:
# 決定 Optimizer，執行完這格，會是 SGD
if OPTIMIZER == "adam":
    opt = optimizers.Adam(lr=0.001,  clipnorm=cfg.MAX_GRAD_NORM)
    cfg.LR= 0.001
if OPTIMIZER == "rmsprop":
    opt = optimizers.RMSprop(lr=0.001,  clipnorm=cfg.MAX_GRAD_NORM)
    cfg.LR= 0.001

if OPTIMIZER == "adagrad":
    opt = optimizers.Adagrad(lr=1.0,  clipnorm=cfg.MAX_GRAD_NORM)
    cfg.LR = 1

#use default is nothing is given
else:
    # create sgd with momentum and gradient clipping
    opt = optimizers.SGD(lr=cfg.LEARNING_RATE, decay=0, momentum=cfg.MOMENTUM,
                         nesterov=False, clipnorm=cfg.MAX_GRAD_NORM)

    cfg.LR = cfg.LEARNING_RATE 


    print("Learning rate: {}".format(cfg.LEARNING_RATE))

In [ ]:
#save config file to log dir
with open(log_dir_name  +'/config.pkl', 'wb') as f:
    pickle.dump(cfg, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#callbacks
cb = []

#add tensorboard callback
tbCallBack = TensorBoard(log_dir=tb_dir, histogram_freq=0, write_graph=True, write_images=True)
cb.append(tbCallBack)

if REDUCELRONPLATEAU:

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,verbose=1, patience=5, min_lr=0.0)

    cb.append(reduce_lr)

if VERBOSE:
    print(squeeze.model.summary())
    
    # will (None, 16848, 12) work in this case? I doubt about it.

In [ ]:
# compile model
squeeze.model.compile(optimizer=opt, loss=[squeeze.loss], metrics=[squeeze.loss_without_regularization, squeeze.bbox_loss, squeeze.class_loss, squeeze.conf_loss])

In [ ]:
# # To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

# tf.autograph.experimental.do_not_convert(
#     squeeze.model.fit_generator(train_generator, 
#                                 epochs=EPOCHS, 
#                                 steps_per_epoch=nbatches_train, 
#                                 callbacks=cb
#     )
# )

In [ ]:
squeeze.model.load_weights("model.hdf5")

In [ ]:
imgs, gts, imgs_only_resized = next(vis_train_generator)

In [ ]:
OldMax = imgs[0].max()
OldMin = imgs[0].min()
NewMax = 255
NewMin = 0

OldRange = (OldMax - OldMin)
NewRange = (NewMax - NewMin)  
img = ((imgs[0] - OldMin) / (OldMax - OldMin) ) * (NewMax - NewMin) + NewMin
img = img.astype(np.uint16) # uint16 is necessary for cvtColor
#https://stackoverflow.com/questions/50319617/error-in-opencv-color-conversion-from-bgr-to-grayscale
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

# draw
plt.figure(figsize=(10,5))
plt.imshow(img)
plt.show()

In [ ]:
result = visualize(squeeze.model, vis_train_generator, cfg)

In [ ]:
plt.figure(figsize=(20,15))
plt.imshow(result[5].astype(np.uint16))
plt.show()

# 遇到一個問題XD 預測(紅色)出來都是一樣的XD 0.07493171